# **<center style="color: green;">Desafío - Transformación y manipulación de datos (Parte II)</center>**

### **Nombre:** Cristian Tobar Morales ###
### **Curso:** Programación con Python para el análisis de datos (G101) ###

### **Descripción**
La **empresa ADL** se dedica a la venta de productos a nivel regional en los Estados Unidos,
con equipos de ventas que operan en diferentes estados. La compañía ha establecido un
sistema de comisiones para incentivar a sus equipos de ventas a preferir la venta de
productos de alto margen antes que los de bajo margen de utilidad.
Como Analista de Datos, le solicitan analizar información sobre las ventas realizadas por
los equipos de ventas de ADL, incluyendo detalles sobre los productos vendidos, los
clientes, las fechas de los pedidos y los plazos de entrega.
Para resolver este desafío deberás cargar datos desde Hojas de Excel del archivo
US_Regional_Sales_Data.xlsx. En base a esto, debe generar una visualización de datos
como la que se muestra en las hojas Reporte1 y Reporte2 del archivo reporte.xlsx.
Para cumplir con lo solicitado, puedes aplicar los siguientes pasos:

1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un
DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando
pd.ExcelFile(archivo).sheet_names)

In [ ]:
# Importar libreria
import pandas as pd

# Ruta del archivo
archivo = '../Desafío_Transformación_Manipulación_Datos/db/'

# Nombre de de las hojas
nombre_hojas = pd.ExcelFile(archivo).sheet_names

#Se crea una diccionario para almacenar los Dataframe
dataframes = {}

#Cargar cada hoja al diccionario
for hoja in nombre_hojas:
    dataframes[hoja] = pd.read_excel(archivo, sheet_name=hoja)

# Mostrar los nombres de los DataFrames
for nombre_hoja, df in dataframes.items():
    print(f'DataFrame para la hoja: {nombre_hoja}')

FileNotFoundError: [Errno 2] No such file or directory: '../db/US_Regional_Sales_Data.xlsx'

2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado
en una nueva variable llamada df_base.

In [1]:
# Los dataframes
df_sales_orders = dataframes['Sales Orders Sheet']
df_customers = dataframes['Customers Sheet']
df_store_locations = dataframes['Store Locations Sheet']
df_products = dataframes['Products Sheet']
df_sales_team = dataframes['Sales Team Sheet']

# Cruzar los dataframes
# 1: Se cruza df_sales_orders con df_customers usando 'CustomerID'
df_base = df_sales_orders.merge(df_customers, on = '_CustomerID', validate = 'many_to_one')

# Luego cruzamos el resultado con df_store_locations usando 'StoreID'
df_base = df_base.merge(df_store_locations, on = '_StoreID', validate = 'many_to_one')

# Luego cruzamos el resultado con df_products usando 'ProductID'
df_base = df_base.merge(df_products, on = '_ProductID', validate = 'many_to_one')

# Finalmente, cruzamos el resultado con df_sales_team usando 'SalesPersonID'
df_base = df_base.merge(df_sales_team, on = '_SalesTeamID', validate = 'many_to_one')

# Mostrar los resultados
df_base.head()


NameError: name 'dataframes' is not defined